<a href="https://colab.research.google.com/github/Gxwthmmmm/Medical-Image-Distortion/blob/training/distorted_image_Training_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

TRAINING THE MODEL

In [ ]:
import numpy as np
import cv2
import os
import shutil
from PIL import Image
import random
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models

# Loading  base image
base_image_path = "brain.png"
if not os.path.exists(base_image_path):
    print(f"Image '{base_image_path}' not found.")
    img_np = None
else:
    img = Image.open(base_image_path).convert("L")
    img_np = np.array(img)

if img_np is not None:
    # Settings
    noise_levels = [10, 20, 30, 40, 50]  # std_dev for Gaussian noise
    num_per_level = 80  # 80 images per class
    img_size = (128, 128)

    # Clean previous dataset
    dataset_dir = "dataset_gaussian"
    if os.path.exists(dataset_dir):
        shutil.rmtree(dataset_dir)

    for i, std_dev in enumerate(noise_levels):
        level_dir = os.path.join(dataset_dir, f"Noise_Level_{i+1}")
        os.makedirs(level_dir, exist_ok=True)

        for j in range(num_per_level):
            noisy_img = img_np + np.random.normal(0, std_dev, img_np.shape)
            noisy_img = np.clip(noisy_img, 0, 255).astype(np.uint8)

            # Resize and save
            resized = cv2.resize(noisy_img, img_size)
            cv2.imwrite(os.path.join(level_dir, f"noisy_{j+1:03}.png"), resized)

    print(" Dataset generated with clean Gaussian noise levels.")

    # Loading data
    batch_size = 16
    datagen = ImageDataGenerator(
        rescale=1./255,
        validation_split=0.2
    )

    train_data = datagen.flow_from_directory(
        dataset_dir,
        target_size=img_size,
        batch_size=batch_size,
        color_mode='grayscale',
        class_mode='categorical',
        subset='training'
    )

    val_data = datagen.flow_from_directory(
        dataset_dir,
        target_size=img_size,
        batch_size=batch_size,
        color_mode='grayscale',
        class_mode='categorical',
        subset='validation'
    )

    # Defining a  deeper CNN
    model = models.Sequential([
        layers.Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 1)),
        layers.MaxPooling2D(2, 2),
        layers.Conv2D(64, (3, 3), activation='relu'),
        layers.MaxPooling2D(2, 2),
        layers.Conv2D(128, (3, 3), activation='relu'),
        layers.Flatten(),
        layers.Dense(64, activation='relu'),
        layers.Dense(train_data.num_classes, activation='softmax')
    ])

    model.compile(optimizer='adam',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])

    print("\n Training model on clean Gaussian noise levels...")
    model.fit(train_data, epochs=10, validation_data=val_data)

    # Evaluation the model
    loss, acc = model.evaluate(val_data)
    print(f"\n Final Validation Accuracy: {acc:.4f}")
else:
    print("Base image could not be loaded. Exiting.")


 Dataset generated with clean Gaussian noise levels.
Found 320 images belonging to 5 classes.
Found 80 images belonging to 5 classes.


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()



 Training model on clean Gaussian noise levels...
Epoch 1/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 13s 511ms/step - accuracy: 0.2086 - loss: 1.7654 - val_accuracy: 0.4000 - val_loss: 1.4237
Epoch 2/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 11s 531ms/step - accuracy: 0.3903 - loss: 1.3086 - val_accuracy: 0.8000 - val_loss: 0.8478
Epoch 3/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 11s 536ms/step - accuracy: 0.8457 - loss: 0.7279 - val_accuracy: 0.8000 - val_loss: 0.3139
Epoch 4/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 12s 589ms/step - accuracy: 0.9488 - loss: 0.1892 - val_accuracy: 1.0000 - val_loss: 0.0898
Epoch 5/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 19s 480ms/step - accuracy: 0.8425 - loss: 0.4028 - val_accuracy: 1.0000 - val_loss: 0.0492
Epoch 6/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 10s 524ms/step - accuracy: 1.0000 - loss: 0.0592 - val_accuracy: 1.0000 - val_loss: 0.0211
Epoch 7/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 10s 529ms/step - accuracy: 1.0000 - loss: 0.0137 - val_accuracy: 1.0000 - val_loss: 0.0067
Epoch 8/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 11s 567m

Checking the accuracy


In [ ]:
# Evaluate the model on validation data
val_loss, val_accuracy = model.evaluate(val_data)
print(f"\nValidation Accuracy: {val_accuracy:.1f}")
print(f"Validation Loss: {val_loss:.4f}")


5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 122ms/step - accuracy: 1.0000 - loss: 7.5662e-04

Validation Accuracy: 1.0
Validation Loss: 0.0008


Testing the trained data to check the noise level of an Image

In [ ]:
import matplotlib.pyplot as plt
from PIL import Image

# Load a new test image (grayscale)
test_img = Image.open("/content/brain_mri_1.png.png").convert("L")
test_img = test_img.resize((128, 128))
test_array = np.array(test_img).astype("float32") / 255.0
test_array = np.expand_dims(test_array, axis=(0, -1))  # Shape: (1, 128, 128, 1)

# Predict the noise level of the image
prediction = model.predict(test_array)
predicted_class = np.argmax(prediction)

# Displays which noise level is the image
plt.imshow(test_img, cmap='gray')
plt.title(f"Predicted Noise Level: {predicted_class + 1}")
plt.axis('off')
plt.show()